<a href="https://colab.research.google.com/github/VitalyGladyshev/LLM-engineering/blob/main/LLM_GLVV_HW14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание: Полноценная мультиагентная система с инструментами

Цель: спроектировать и реализовать систему из нескольких LLM‑агентов, которые координируются, обмениваются сообщениями и вызывают инструменты для решения реальной задачи end‑to‑end.


## Требования (обязательные)
- Роли агентов (минимум 3) с чётко разделёнными обязанностями и разными наборами инструментов для каждой роли.
- Коммуникация: шина сообщений или адресные сообщения (ask_agent / reply) с логами входящих/исходящих сообщений.
- Инструменты (минимум 3) с реальными побочными эффектами: например, работа с файлами/кодом/HTTP/данными/оценкой.
- SGR/Structured Output: схемы действий/сообщений (UseTool/AskAgent/Reply/Finish) с валидацией.
- Оркестрация: централизованный планировщик или децентрализованная логика (обосновать выбор).
- Демонстрация: один целевой сценарий, доведённый до finish, с логами и артефактами (код/отчёт/файлы).


## Оценивание (10 баллов)
- **3 балла — Дизайн агентов и протокола**
  - Чёткие роли и разграничение ответственности (1)
  - Разные доступные инструменты у разных ролей (1)
  - SGR/схемы и валидация structured output (1)
- **5 баллов — Мультиагентность и взаимодействие**
  - Координация и обмен сообщениями между агентами (2)
  - Реальные вызовы инструментов и побочные эффекты (2)
  - Обработка ошибок/ретраи/эскалация (1)
- **1 балл — Качество кода и отчёта**
  - Читаемость, структура, документация и воспроизводимость
- **1 балл — Бонус: >5 уникальных инструментов**
  - Разные по назначению функции, а не дубликаты


## Идеи задач
- Кодогенерация по спецификации: Planner → Coder → Tester → Reviewer.
- Data/RAG пайплайн: Ingestor → Indexer → Analyst → Evaluator.
- Веб‑интеграция: Researcher (web/http), Summarizer, Reporter, Verifier.

## Как сдавать
- Ноутбук с кодом агентов, инструментов и оркестратора.
- README с архитектурой (роли, взаимодействие), списком инструментов, схемами SGR, логами одного прогона, ограничениями и метриками.
- (Опционально) Короткое видео/гиф ≤3 мин с успешным прогоном.

## Штрафы
- − до 2: нет логов или невоспроизводимо
- − до 2: инструменты не делают реальных действий
- − до 1: смешение ролей/инструментов без обоснования


## Стартовые подсказки (не обязательно)
- Начните с 3 ролей и простого набора инструментов; затем добавляйте Reviewer/Verifier/Deployer.
- Схемы SGR: определите классы UseTool/AskAgent/Reply/Finish.
- Ограничьте инструменты по ролям на уровне оркестратора (enforcement).
- Логи: печатайте inbox/outbox, действие (JSON), результаты инструментов (stdout/error), diff при изменении файлов.
- Введите ретраи и сообщения «поясни ошибку», если JSON от модели невалиден или инструмент упал.


In [11]:
from google.colab import userdata

api_key = userdata.get('OPENAI_API_KEY')
print("Ключ получен!")

Ключ получен!


In [12]:
# 1) Настройка OpenAI API (обязательное использование LLM)

import os
from typing import Optional

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

DEFAULT_MODEL = "openai/gpt-5-mini"  # можно поменять на совместимую с JSON schema
openai_client = None

if os.environ.get("OPENAI_API_KEY"):
    try:
        from openai import OpenAI
        openai_client = OpenAI(
            base_url="https://api.vsegpt.ru/v1",
            api_key=os.environ["OPENAI_API_KEY"],
        )
        print("OpenAI API готов. Модель:", DEFAULT_MODEL)
    except Exception as e:
        raise RuntimeError(f"Не удалось инициализировать OpenAI API: {e}")
else:
    raise RuntimeError("OPENAI_API_KEY не найден. Установите ключ в окружении: export OPENAI_API_KEY=...")


OpenAI API готов. Модель: openai/gpt-5-mini


In [13]:
# 2) вызов LLM с JSON Schema

from pydantic import BaseModel
from typing import Optional
import json

def call_llm_with_schema(
    prompt: str,
    response_schema: type[BaseModel],
    system_prompt: Optional[str] = None,
    strict: bool = True
) -> BaseModel:
    if openai_client is None:
        raise RuntimeError("OpenAI API клиент не инициализирован.")

    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})

    # Получаем схему от Pydantic
    schema_dict = response_schema.model_json_schema()

    # Для strict mode: все поля в required + additionalProperties: false
    if strict and "properties" in schema_dict:
        schema_dict["required"] = list(schema_dict["properties"].keys())
        schema_dict["additionalProperties"] = False

    try:
        resp = openai_client.chat.completions.create(
            model=DEFAULT_MODEL,
            messages=messages,
            response_format={
                "type": "json_schema",
                "json_schema": {
                    "name": response_schema.__name__,
                    "schema": schema_dict,
                    "strict": strict
                },
            },
            temperature=0.2,
            max_tokens=2000,
        )
        raw = resp.choices[0].message.content or ""

    except Exception as api_error:
        print(f"Structured output не сработал: {api_error}")
        # Фолбэк на обычный JSON
        resp = openai_client.chat.completions.create(
            model=DEFAULT_MODEL,
            messages=messages,
            response_format={"type": "json_object"},
            temperature=0.2,
            max_tokens=2000,
        )
        raw = resp.choices[0].message.content or ""

    # Извлечение и парсинг JSON
    def extract_json(s: str) -> str:
        s = s.strip()
        for prefix in ["```json", "```"]:
            if s.startswith(prefix):
                s = s[len(prefix):].strip()
        if s.endswith("```"):
            s = s[:-3].strip()

        first, last = s.find('{'), s.rfind('}')
        if first != -1 and last > first:
            return s[first:last+1]
        return s

    parsed_json = extract_json(raw)
    return response_schema.model_validate_json(parsed_json)


In [14]:
# 3) SGR: действия агента и сообщения

from typing import Dict, Any, List, Literal
from pydantic import BaseModel, Field, ConfigDict

class AgentActionFlat(BaseModel):
    """
    Плоская структура ответа агента.
    tool_args передаётся как JSON-строка для совместимости со strict mode.
    """
    action: Literal["ask_agent", "use_tool", "reply", "finish"]

    # Поля для ask_agent
    target: Literal["planner", "coder", "tester", "reviewer", "none"] = "none"
    message: str = ""

    # Поля для use_tool (args как JSON-строка!)
    tool_name: str = ""
    tool_args_json: str = ""  # JSON-строка вместо Dict

    # Поля для reply
    content: str = ""

    # Поля для finish
    summary: str = ""

    model_config = ConfigDict(
        extra="forbid"  # Запрещает лишние поля
    )

    def get_tool_args(self) -> Dict[str, Any]:
        """Парсит tool_args_json в словарь"""
        if not self.tool_args_json:
            return {}
        try:
            return json.loads(self.tool_args_json)
        except json.JSONDecodeError:
            return {}

class BusMessage(BaseModel):
    sender: str
    recipient: str
    content: str
    timestamp: float = Field(default_factory=lambda: __import__('time').time())


In [15]:
# 4) Workspace и инструменты (расширенный набор)

from io import StringIO
import sys
import traceback
import difflib
import json
import re

class Workspace:
    def __init__(self):
        self.files: Dict[str, str] = {}
        self.memory: Dict[str, Any] = {}
        self.test_results: Dict[str, Dict] = {}
        self.code_reviews: Dict[str, Dict] = {}

    # Инструменты для Coder
    def store_code(self, filename: str, code: str) -> Dict[str, Any]:
        """Сохраняет код в файл с генерацией diff"""
        prev = self.files.get(filename, "")
        self.files[filename] = code
        diff = ""
        if prev:
            diff = "\n".join(difflib.unified_diff(
                prev.splitlines(),
                code.splitlines(),
                fromfile=f"prev:{filename}",
                tofile=f"new:{filename}",
                lineterm=""
            ))
        return {
            "message": f"stored {filename}",
            "filename": filename,
            "chars": len(code),
            "lines": len(code.splitlines()),
            "diff": diff[:2000] if diff else "no previous version"
        }

    def read_code(self, filename: str) -> Dict[str, Any]:
        """Читает код из файла"""
        code = self.files.get(filename, "")
        return {
            "filename": filename,
            "code": code,
            "exists": bool(code),
            "lines": len(code.splitlines()) if code else 0
        }

    # Инструменты для Tester
    def run_python(self, code: str) -> Dict[str, Any]:
        """Выполняет Python код в изолированном окружении"""
        captured = StringIO()
        old_stdout = sys.stdout
        sys.stdout = captured
        result = {"stdout": "", "error": "", "success": False}

        try:
            # Ограниченное окружение для безопасности
            allowed_globals = {
                "__builtins__": {
                    "print": print,
                    "range": range,
                    "len": len,
                    "sum": sum,
                    "min": min,
                    "max": max,
                    "abs": abs,
                    "list": list,
                    "dict": dict,
                    "str": str,
                    "int": int,
                    "float": float,
                    "bool": bool,
                    "set": set,
                    "tuple": tuple,
                    "type": type,
                    "isinstance": isinstance,
                    "issubclass": issubclass,
                    "hasattr": hasattr,
                    "getattr": getattr,
                    "setattr": setattr,
                    "delattr": delattr,
                }
            }

            exec_locals = self.memory.copy()
            exec(code, allowed_globals, exec_locals)

            # Обновляем память workspace
            self.memory.update(exec_locals)

            result["stdout"] = captured.getvalue()
            result["success"] = True
        except Exception as e:
            result["error"] = f"{type(e).__name__}: {str(e)}\n{traceback.format_exc()}"
            result["success"] = False
        finally:
            sys.stdout = old_stdout

        return result

    def run_tests(self, filename: str, test_code: str) -> Dict[str, Any]:
        """Запускает тесты для указанного файла"""
        code = self.files.get(filename, "")
        if not code:
            return {"error": f"File {filename} not found", "passed": False}

        # Комбинируем основной код и тесты
        combined_code = f"""
# Main code
{code}

# Test code
{test_code}
"""

        result = self.run_python(combined_code)
        passed = result["success"] and not result["error"]

        # Сохраняем результаты тестов
        test_key = f"{filename}_tests"
        self.test_results[test_key] = {
            "passed": passed,
            "stdout": result["stdout"],
            "error": result["error"],
            "timestamp": __import__('time').time()
        }

        return {
            "passed": passed,
            "file": filename,
            "stdout": result["stdout"][:500],
            "error": result["error"][:500] if result["error"] else "",
            "success": result["success"]
        }

    # Инструменты для Reviewer
    def analyze_code_quality(self, filename: str) -> Dict[str, Any]:
        """Анализирует качество кода по различным метрикам"""
        code_info = self.read_code(filename)
        if not code_info["exists"]:
            return {"error": f"File {filename} not found", "quality_score": 0}

        code = code_info["code"]
        lines = code.splitlines()

        # Простой анализ качества кода
        has_docstrings = any('"""' in line or "'''" in line for line in lines)
        has_comments = any(line.strip().startswith('#') and not line.strip().startswith('# noqa') for line in lines if line.strip())
        function_count = len([line for line in lines if line.strip().startswith('def ')])
        class_count = len([line for line in lines if line.strip().startswith('class ')])
        complexity_score = len([line for line in lines if any(op in line for op in ['if ', 'else', 'for ', 'while ', 'try', 'except'])])

        # Оценка качества (упрощенная)
        quality_score = 0
        if has_docstrings: quality_score += 25
        if has_comments: quality_score += 25
        if function_count > 0 or class_count > 0: quality_score += 25
        if complexity_score < 10: quality_score += 25  # Низкая сложность - хорошо для простых задач

        review = {
            "filename": filename,
            "lines": code_info["lines"],
            "functions": function_count,
            "classes": class_count,
            "has_docstrings": has_docstrings,
            "has_comments": has_comments,
            "complexity_score": complexity_score,
            "quality_score": min(100, quality_score),
            "recommendations": []
        }

        if not has_docstrings:
            review["recommendations"].append("Добавьте docstrings к функциям и классам")
        if not has_comments:
            review["recommendations"].append("Добавьте комментарии для объяснения сложной логики")
        if complexity_score > 15:
            review["recommendations"].append("Рассмотрите возможность разделения сложных функций")

        # Сохраняем результаты анализа
        self.code_reviews[filename] = review

        return review

    def generate_report(self) -> Dict[str, Any]:
        """Генерирует итоговый отчет по всем артефактам"""
        report = {
            "files": list(self.files.keys()),
            "test_results": self.test_results,
            "code_reviews": self.code_reviews,
            "summary": {
                "total_files": len(self.files),
                "tested_files": len(self.test_results),
                "reviewed_files": len(self.code_reviews),
                "overall_quality": 0
            }
        }

        # Рассчитываем общий показатель качества
        quality_scores = [review["quality_score"] for review in self.code_reviews.values()]
        if quality_scores:
            report["summary"]["overall_quality"] = sum(quality_scores) / len(quality_scores)

        return report



In [16]:
# 5) Шина сообщений и шаблон агента

from collections import deque
import time

class MessageBus:
    def __init__(self):
        self.queue = deque()
        self.history: List[BusMessage] = []
        self.log_file = "agent_communication.log"

    def send(self, msg: BusMessage):
        """Отправляет сообщение в шину и логирует"""
        self.queue.append(msg)
        self.history.append(msg)

        # Логируем сообщение в файл
        with open(self.log_file, "a", encoding="utf-8") as f:
            timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(msg.timestamp))
            f.write(f"[{timestamp}] {msg.sender} → {msg.recipient}: {msg.content}\n")

        print(f"[{msg.sender} → {msg.recipient}]: {msg.content[:100]}...")

    def receive_for(self, recipient: str) -> List[BusMessage]:
        """Получает сообщения для конкретного получателя"""
        msgs = []
        remaining = deque()

        for msg in self.queue:
            if msg.recipient in (recipient, "broadcast"):
                msgs.append(msg)
            else:
                remaining.append(msg)

        self.queue = remaining
        return msgs

    def get_last_messages(self, count: int = 10) -> List[BusMessage]:
        """Возвращает последние сообщения из истории"""
        return self.history[-count:]

ws = Workspace()
bus = MessageBus()


In [17]:
# 6) Инструменты с расширенной функциональностью

class Tools:
    def __init__(self, ws: Workspace):
        self.ws = ws

    def call(self, tool_name: str, args: Dict[str, Any]) -> Dict[str, Any]:
        """Вызывает инструмент по имени с аргументами"""
        try:
            if tool_name == "store_code":
                return self.ws.store_code(args.get("filename", ""), args.get("code", ""))
            elif tool_name == "read_code":
                return self.ws.read_code(args.get("filename", ""))
            elif tool_name == "run_python":
                return self.ws.run_python(args.get("code", ""))
            elif tool_name == "run_tests":
                return self.ws.run_tests(args.get("filename", ""), args.get("test_code", ""))
            elif tool_name == "analyze_code_quality":
                return self.ws.analyze_code_quality(args.get("filename", ""))
            elif tool_name == "generate_report":
                return self.ws.generate_report()
            else:
                return {"error": f"Unknown tool: {tool_name}", "available_tools": self.get_available_tools()}
        except Exception as e:
            return {"error": f"Tool execution failed: {str(e)}", "exception": str(e)}

    def get_available_tools(self) -> List[str]:
        """Возвращает список доступных инструментов"""
        return [
            "store_code", "read_code", "run_python",
            "run_tests", "analyze_code_quality", "generate_report"
        ]

tools = Tools(ws)


In [18]:
# 7) Расширенные агенты с конкретными ролями

class LlmAgent:
    def __init__(self, name: str, system_role: str, allowed_tools: List[str], description: str):
        self.name = name
        self.system_role = system_role
        self.allowed_tools = allowed_tools
        self.description = description

    def get_context(self, goal: str) -> str:
        recent_messages = bus.get_last_messages(10)
        history_text = "\n".join([
            f"[{msg.sender} → {msg.recipient}]: {msg.content[:200]}"
            for msg in recent_messages
        ])

        files_info = f"Файлы: {list(ws.files.keys())}" if ws.files else "Файлов пока нет"

        return f"""ЦЕЛЬ: {goal}

ИСТОРИЯ:
{history_text}

WORKSPACE: {files_info}
ВАШИ ИНСТРУМЕНТЫ: {', '.join(self.allowed_tools) if self.allowed_tools else 'нет'}"""

    def decide(self, goal: str) -> AgentActionFlat:
        context = self.get_context(goal)

        # Примеры для каждого действия
        examples = '''
ПРИМЕРЫ JSON (копируйте формат точно!):

1. Попросить другого агента:
{"action": "ask_agent", "target": "coder", "message": "Напиши класс Calculator", "tool_name": "", "tool_args_json": "", "content": "", "summary": ""}

2. Использовать инструмент store_code:
{"action": "use_tool", "target": "none", "message": "", "tool_name": "store_code", "tool_args_json": "{\\"filename\\": \\"calculator.py\\", \\"code\\": \\"class Calculator:\\n    pass\\"}", "content": "", "summary": ""}

3. Использовать инструмент run_tests:
{"action": "use_tool", "target": "none", "message": "", "tool_name": "run_tests", "tool_args_json": "{\\"filename\\": \\"calculator.py\\", \\"test_code\\": \\"calc = Calculator()\\nassert calc.add(2,3) == 5\\"}", "content": "", "summary": ""}

4. Ответить/сообщить о прогрессе:
{"action": "reply", "target": "none", "message": "", "tool_name": "", "tool_args_json": "", "content": "Код готов, передаю тестировщику", "summary": ""}

5. Завершить задачу:
{"action": "finish", "target": "none", "message": "", "tool_name": "", "tool_args_json": "", "content": "", "summary": "Калькулятор реализован, протестирован и проверен"}
'''

        system_prompt = f"""Вы — {self.system_role}.
Роль: {self.description}
Доступные инструменты: {', '.join(self.allowed_tools) if self.allowed_tools else 'нет'}

КРИТИЧЕСКИ ВАЖНО:
- Возвращайте ТОЛЬКО JSON, без текста до или после
- ВСЕ 7 полей обязательны в каждом ответе
- tool_args_json — это СТРОКА с экранированным JSON внутри
- Неиспользуемые поля заполняйте пустой строкой "" или "none" для target

{examples}"""

        prompt = f"""{context}

Что вы делаете? Верните JSON с вашим действием:"""

        try:
            return call_llm_with_schema(
                prompt=prompt,
                response_schema=AgentActionFlat,
                system_prompt=system_prompt,
                strict=True
            )
        except Exception as e:
            print(f"[{self.name}] Ошибка LLM: {e}")
            return self._default_action(goal)

    def _default_action(self, goal: str) -> AgentActionFlat:
        """Действие по умолчанию при ошибке"""
        if self.name == "planner":
            return AgentActionFlat(
                action="ask_agent", target="coder",
                message=f"Реализуй: {goal[:150]}"
            )
        elif self.name == "coder":
            return AgentActionFlat(action="reply", content="Жду спецификацию от planner")
        elif self.name == "tester":
            return AgentActionFlat(action="ask_agent", target="coder", message="Нужен код для тестов")
        else:
            return AgentActionFlat(action="reply", content="Ожидаю данные")


In [19]:
# 8) Оркестратор с полной логикой

class Orchestrator:
    def __init__(self):
        self.agents = {
            "planner": LlmAgent(
                name="planner",
                system_role="Архитектор и Планировщик",
                allowed_tools=[],
                description="Анализирует требования и координирует работу команды"
            ),
            "coder": LlmAgent(
                name="coder",
                system_role="Разработчик",
                allowed_tools=["store_code", "read_code", "run_python"],
                description="Пишет чистый, документированный Python код"
            ),
            "tester": LlmAgent(
                name="tester",
                system_role="Тестировщик",
                allowed_tools=["read_code", "run_tests"],
                description="Пишет и запускает unit-тесты"
            ),
            "reviewer": LlmAgent(
                name="reviewer",
                system_role="Ревьювер",
                allowed_tools=["read_code", "analyze_code_quality", "generate_report"],
                description="Проводит code review и генерирует отчёты"
            )
        }
        self.allowed_tools = {name: agent.allowed_tools for name, agent in self.agents.items()}
        self.execution_log = []

    def log_step(self, round_num: int, agent_name: str, action: AgentActionFlat, result: Any = None):
        log_entry = {
            "round": round_num,
            "agent": agent_name,
            "action_type": action.action,
            "timestamp": time.time()
        }
        self.execution_log.append(log_entry)

    def process_action(self, agent_name: str, action: AgentActionFlat, goal: str) -> bool:
        """Обрабатывает действие агента"""

        if action.action == "ask_agent":
            if action.target not in self.agents or action.target == "none":
                print(f"[{agent_name}] Неверный target: {action.target}")
                return False

            bus.send(BusMessage(sender=agent_name, recipient=action.target, content=action.message))
            print(f"📨 [{agent_name} → {action.target}]: {action.message[:100]}...")
            return False

        elif action.action == "use_tool":
            if action.tool_name not in self.allowed_tools.get(agent_name, []):
                print(f"[{agent_name}] Нет доступа к инструменту: {action.tool_name}")
                bus.send(BusMessage(
                    sender="orchestrator", recipient=agent_name,
                    content=f"Инструмент '{action.tool_name}' недоступен. Ваши: {self.allowed_tools.get(agent_name, [])}"
                ))
                return False

            # Парсим аргументы из JSON-строки
            tool_args = action.get_tool_args()

            result = tools.call(action.tool_name, tool_args)
            result_str = json.dumps(result, ensure_ascii=False)[:400]

            bus.send(BusMessage(
                sender=agent_name, recipient="broadcast",
                content=f"[{action.tool_name}] Результат: {result_str}"
            ))
            print(f"[{agent_name}] {action.tool_name}: {result_str[:150]}...")
            return False

        elif action.action == "reply":
            bus.send(BusMessage(sender=agent_name, recipient="broadcast", content=action.content))
            print(f"[{agent_name}]: {action.content[:150]}...")
            return False

        elif action.action == "finish":
            print(f"[{agent_name}] ЗАВЕРШЕНО: {action.summary}")
            bus.send(BusMessage(sender=agent_name, recipient="broadcast", content=f"FINISH: {action.summary}"))

            # Генерируем финальный отчёт
            report = ws.generate_report()
            print(f"\nИТОГОВЫЙ ОТЧЁТ:\n{json.dumps(report, ensure_ascii=False, indent=2)}")
            return True

        return False

    def step(self, goal: str, order: List[str], round_num: int) -> bool:
        print(f"\n{'='*50}\nРАУНД {round_num}\n{'='*50}")

        for agent_name in order:
            print(f"\n🔹 Агент: {agent_name.upper()}")

            inbox = bus.receive_for(agent_name)
            if inbox:
                print(f"Входящие ({len(inbox)}):")
                for msg in inbox[-2:]:
                    print(f"   [{msg.sender}]: {msg.content[:80]}...")

            action = self.agents[agent_name].decide(goal)

            # ИСПРАВЛЕНО: action.action вместо action.step.action
            print(f"   Действие: {action.action}")

            self.log_step(round_num, agent_name, action)

            if self.process_action(agent_name, action, goal):
                return True

        return False

    def run(self, goal: str, max_rounds: int = 12):
        print(f"ЗАПУСК СИСТЕМЫ\nЦель: {goal[:200]}...\n")

        bus.send(BusMessage(sender="orchestrator", recipient="broadcast", content=f"СТАРТ. Цель: {goal[:150]}"))

        for round_num in range(1, max_rounds + 1):
            if self.step(goal, ["planner", "coder", "tester", "reviewer"], round_num):
                break
        else:
            print(f"\nЛимит раундов ({max_rounds}) достигнут")

        self.print_final_artifacts()

    def print_final_artifacts(self):
        print(f"\n{'='*50}\nФИНАЛЬНЫЕ АРТЕФАКТЫ\n{'='*50}")

        for filename, code in ws.files.items():
            print(f"\n📄 {filename}:\n{'-'*40}\n{code}\n{'-'*40}")

        if ws.test_results:
            print("\nТесты:")
            for name, res in ws.test_results.items():
                status = "Соответствует" if res.get("passed") else "Провалено"
                print(f"   {status} {name}")

In [20]:
GOAL = """
Реализовать класс Calculator для математических операций:
- Метод add(x, y): сложение двух чисел
- Метод subtract(x, y): вычитание
- Метод multiply(x, y): умножение
- Метод divide(x, y): деление с проверкой деления на ноль
- Метод power(x, y): возведение в степень
- Метод sqrt(x): квадратный корень с проверкой отрицательного числа

Требования:
- Код должен быть чистым и хорошо документированным
- Класс должен обрабатывать ошибки (деление на ноль, отрицательные числа для корня)
- Написать unit-тесты для всех методов
- Провести code review и сгенерировать отчет
"""

print("="*80)
print("Запуск мультиагентной системы для кодогенерации")
print("="*80)
print(f"Задача:\n{GOAL}")
print("="*80)

# Инициализация и запуск системы
orchestrator = Orchestrator()
orchestrator.run(goal=GOAL, max_rounds=12)

print("\n" + "="*80)
print("Работа системы кодогенерации завершена")
print("Все артефакты сохранены в рабочую директорию.")
print("="*80)

Запуск мультиагентной системы для кодогенерации
Задача:

Реализовать класс Calculator для математических операций:
- Метод add(x, y): сложение двух чисел
- Метод subtract(x, y): вычитание
- Метод multiply(x, y): умножение  
- Метод divide(x, y): деление с проверкой деления на ноль
- Метод power(x, y): возведение в степень
- Метод sqrt(x): квадратный корень с проверкой отрицательного числа

Требования:
- Код должен быть чистым и хорошо документированным
- Класс должен обрабатывать ошибки (деление на ноль, отрицательные числа для корня)
- Написать unit-тесты для всех методов
- Провести code review и сгенерировать отчет

ЗАПУСК СИСТЕМЫ
Цель: 
Реализовать класс Calculator для математических операций:
- Метод add(x, y): сложение двух чисел
- Метод subtract(x, y): вычитание
- Метод multiply(x, y): умножение  
- Метод divide(x, y): деление с ...

[orchestrator → broadcast]: СТАРТ. Цель: 
Реализовать класс Calculator для математических операций:
- Метод add(x, y): сложение ...

РАУНД 1

🔹 Аген

## Заключение

Система демонстрирует end-to-end решение реальной задачи разработки ПО с автоматическим обеспечением качества кода. Архитектура легко расширяется для добавления новых агентов и инструментов, что делает ее основой для построения сложных мультиагентных систем в production-средах.

Архитектура из 4 агентов с четким разделением обязанностей:

- Planner — анализирует требования и создает техническое задание
- Coder — генерирует чистый и документированный код
- Tester — пишет и запускает unit-тесты с обработкой ошибок
- Reviewer — проводит code review и формирует итоговый отчет

Реализовано:

- Сохранение/чтение кода с diff-анализом
- Безопасное выполнение Python-кода
- Автоматическое тестирование
- Анализ качества кода по метрикам
- Генерация финального отчета

In [21]:
import json
import time
import base64
import traceback
import re
from io import StringIO
import sys
from enum import Enum
from collections import deque
from typing import Dict, Any, List, Literal, Optional, Union
from pydantic import BaseModel, Field, ConfigDict

# ============================================================================
# 3) СХЕМЫ ДАННЫХ (ПЛОСКАЯ СТРУКТУРА С BASE64)
# ============================================================================

class AgentActionFlat(BaseModel):
    """
    Плоская структура действия агента.
    Код передаётся в base64 для избежания проблем с экранированием.
    """
    action: Literal["ask_agent", "use_tool", "reply", "finish"]

    # Для ask_agent
    target: Literal["planner", "coder", "tester", "reviewer", "none"] = "none"
    message: str = ""

    # Для use_tool - простые поля вместо вложенного JSON
    tool_name: str = ""
    filename: str = ""
    code_base64: str = ""      # Код в base64
    test_code_base64: str = "" # Тестовый код в base64

    # Для reply/finish
    content: str = ""
    summary: str = ""

    model_config = ConfigDict(extra="forbid")

    def get_decoded_code(self) -> str:
        """Декодирует код из base64"""
        if not self.code_base64:
            return ""
        try:
            # Пробуем декодировать base64
            return base64.b64decode(self.code_base64).decode('utf-8')
        except Exception:
            # Если не base64, возвращаем как есть (фолбэк)
            return self.code_base64

    def get_decoded_test_code(self) -> str:
        """Декодирует тестовый код из base64"""
        if not self.test_code_base64:
            return ""
        try:
            return base64.b64decode(self.test_code_base64).decode('utf-8')
        except Exception:
            return self.test_code_base64


class BusMessage(BaseModel):
    """Сообщение между агентами"""
    sender: str
    recipient: str
    content: str
    timestamp: float = Field(default_factory=time.time)


class WorkflowState(Enum):
    """Состояния рабочего процесса"""
    PLANNING = "planning"
    CODING = "coding"
    TESTING = "testing"
    REVIEWING = "reviewing"
    DONE = "done"

# ============================================================================
# 4) ВЫЗОВ LLM С ОБРАБОТКОЙ ОШИБОК
# ============================================================================

def call_llm_with_schema(
    prompt: str,
    response_schema: type[BaseModel],
    system_prompt: Optional[str] = None,
    strict: bool = True,
    max_retries: int = 2
) -> BaseModel:
    """Вызов LLM с JSON Schema и retry логикой"""

    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})

    # Формируем схему
    schema_dict = response_schema.model_json_schema()
    if "properties" in schema_dict:
        schema_dict["required"] = list(schema_dict["properties"].keys())
        schema_dict["additionalProperties"] = False

    last_error = None

    for attempt in range(max_retries + 1):
        try:
            # Пробуем structured output
            resp = openai_client.chat.completions.create(
                model=DEFAULT_MODEL,
                messages=messages,
                response_format={
                    "type": "json_schema",
                    "json_schema": {
                        "name": response_schema.__name__,
                        "schema": schema_dict,
                        "strict": strict
                    },
                },
                temperature=0.1,
                max_tokens=4000,
            )
            raw = resp.choices[0].message.content or ""

            if not raw.strip():
                raise ValueError("Пустой ответ от LLM")

            # Извлекаем JSON
            parsed_json = extract_json(raw)
            return response_schema.model_validate_json(parsed_json)

        except Exception as e:
            last_error = e
            print(f"⚠️ Попытка {attempt + 1}/{max_retries + 1} не удалась: {e}")

            if attempt < max_retries:
                # Пробуем без strict mode
                try:
                    resp = openai_client.chat.completions.create(
                        model=DEFAULT_MODEL,
                        messages=messages,
                        response_format={"type": "json_object"},
                        temperature=0.1,
                        max_tokens=4000,
                    )
                    raw = resp.choices[0].message.content or ""
                    if raw.strip():
                        parsed_json = extract_json(raw)
                        return response_schema.model_validate_json(parsed_json)
                except Exception as inner_e:
                    print(f"⚠️ Фолбэк тоже не удался: {inner_e}")
                    continue

    raise last_error or ValueError("Не удалось получить ответ от LLM")


def extract_json(s: str) -> str:
    """Извлекает JSON из строки"""
    s = s.strip()

    # Удаляем markdown
    if s.startswith("```json"):
        s = s[7:]
    if s.startswith("```"):
        s = s[3:]
    if s.endswith("```"):
        s = s[:-3]
    s = s.strip()

    # Ищем JSON объект
    first = s.find('{')
    last = s.rfind('}')
    if first != -1 and last > first:
        candidate = s[first:last+1]
        try:
            json.loads(candidate)
            return candidate
        except json.JSONDecodeError:
            pass

    # Пробуем как есть
    try:
        json.loads(s)
        return s
    except json.JSONDecodeError:
        pass

    raise ValueError(f"Не удалось извлечь JSON: {s[:200]}...")


def call_llm_simple(prompt: str, system_prompt: str = "") -> str:
    """Простой вызов LLM без JSON schema - для генерации кода"""
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})

    resp = openai_client.chat.completions.create(
        model=DEFAULT_MODEL,
        messages=messages,
        temperature=0.2,
        max_tokens=4000,
    )

    return resp.choices[0].message.content or ""

# ============================================================================
# 5) WORKSPACE - ХРАНИЛИЩЕ ФАЙЛОВ И РЕЗУЛЬТАТОВ
# ============================================================================

class Workspace:
    def __init__(self):
        self.files: Dict[str, str] = {}
        self.memory: Dict[str, Any] = {}
        self.test_results: Dict[str, Dict] = {}
        self.code_reviews: Dict[str, Dict] = {}

    def store_code(self, filename: str, code: str) -> Dict[str, Any]:
        """Сохраняет код в файл"""
        if not filename or not code:
            return {"error": "Пустое имя файла или код", "success": False}

        self.files[filename] = code
        return {
            "success": True,
            "message": f"Файл {filename} сохранён",
            "filename": filename,
            "lines": len(code.splitlines()),
            "chars": len(code)
        }

    def read_code(self, filename: str) -> Dict[str, Any]:
        """Читает код из файла"""
        code = self.files.get(filename, "")
        return {
            "filename": filename,
            "code": code,
            "exists": bool(code),
            "lines": len(code.splitlines()) if code else 0
        }

    def run_python(self, code: str) -> Dict[str, Any]:
        """Выполняет Python код"""
        captured = StringIO()
        old_stdout = sys.stdout
        sys.stdout = captured
        result = {"stdout": "", "error": "", "success": False}

        try:
            exec_globals = {"__builtins__": __builtins__}
            exec_locals = {}
            exec(code, exec_globals, exec_locals)
            result["stdout"] = captured.getvalue()
            result["success"] = True
        except Exception as e:
            result["error"] = f"{type(e).__name__}: {e}"
            result["success"] = False
        finally:
            sys.stdout = old_stdout

        return result

    def run_tests(self, filename: str, test_code: str) -> Dict[str, Any]:
        """Запускает тесты для файла"""
        main_code = self.files.get(filename, "")
        if not main_code:
            return {"error": f"Файл {filename} не найден", "passed": False, "success": False}

        if not test_code:
            return {"error": "Тестовый код пустой", "passed": False, "success": False}

        combined = f"{main_code}\n\n# === TESTS ===\n{test_code}"
        result = self.run_python(combined)

        passed = result["success"] and not result["error"]

        self.test_results[filename] = {
            "passed": passed,
            "stdout": result["stdout"],
            "error": result["error"],
            "timestamp": time.time()
        }

        return {
            "passed": passed,
            "success": result["success"],
            "file": filename,
            "stdout": result["stdout"][:500],
            "error": result["error"][:500] if result["error"] else ""
        }

    def analyze_code_quality(self, filename: str) -> Dict[str, Any]:
        """Анализирует качество кода"""
        info = self.read_code(filename)
        if not info["exists"]:
            return {"error": f"Файл {filename} не найден", "quality_score": 0}

        code = info["code"]
        lines = code.splitlines()

        has_docstrings = '"""' in code or "'''" in code
        has_comments = any(line.strip().startswith('#') for line in lines if line.strip())
        functions = len([l for l in lines if l.strip().startswith('def ')])
        classes = len([l for l in lines if l.strip().startswith('class ')])

        score = 0
        if has_docstrings: score += 30
        if has_comments: score += 20
        if functions > 0 or classes > 0: score += 30
        if len(lines) > 10: score += 20

        review = {
            "filename": filename,
            "quality_score": min(100, score),
            "lines": info["lines"],
            "functions": functions,
            "classes": classes,
            "has_docstrings": has_docstrings,
            "has_comments": has_comments,
            "recommendations": []
        }

        if not has_docstrings:
            review["recommendations"].append("Добавьте docstrings")
        if not has_comments:
            review["recommendations"].append("Добавьте комментарии")

        self.code_reviews[filename] = review
        return review

    def generate_report(self) -> Dict[str, Any]:
        """Генерирует итоговый отчёт"""
        return {
            "files": list(self.files.keys()),
            "test_results": self.test_results,
            "code_reviews": self.code_reviews,
            "summary": {
                "total_files": len(self.files),
                "tested": len(self.test_results),
                "reviewed": len(self.code_reviews)
            }
        }

# ============================================================================
# 6) ШИНА СООБЩЕНИЙ
# ============================================================================

class MessageBus:
    def __init__(self):
        self.queue = deque()
        self.history: List[BusMessage] = []

    def send(self, msg: BusMessage):
        """Отправляет сообщение"""
        self.queue.append(msg)
        self.history.append(msg)
        print(f"📨 [{msg.sender} → {msg.recipient}]: {msg.content[:80]}...")

    def receive_for(self, recipient: str) -> List[BusMessage]:
        """Получает сообщения для получателя"""
        msgs = []
        remaining = deque()

        for msg in self.queue:
            if msg.recipient in (recipient, "broadcast"):
                msgs.append(msg)
            else:
                remaining.append(msg)

        self.queue = remaining
        return msgs

    def get_last_messages(self, count: int = 10) -> List[BusMessage]:
        """Возвращает последние сообщения"""
        return self.history[-count:]

# ============================================================================
# 7) ИНСТРУМЕНТЫ
# ============================================================================

class Tools:
    def __init__(self, ws: Workspace):
        self.ws = ws

    def call(self, tool_name: str, **kwargs) -> Dict[str, Any]:
        """Вызов инструмента"""
        try:
            if tool_name == "store_code":
                return self.ws.store_code(kwargs.get("filename", ""), kwargs.get("code", ""))
            elif tool_name == "read_code":
                return self.ws.read_code(kwargs.get("filename", ""))
            elif tool_name == "run_python":
                return self.ws.run_python(kwargs.get("code", ""))
            elif tool_name == "run_tests":
                return self.ws.run_tests(kwargs.get("filename", ""), kwargs.get("test_code", ""))
            elif tool_name == "analyze_code_quality":
                return self.ws.analyze_code_quality(kwargs.get("filename", ""))
            elif tool_name == "generate_report":
                return self.ws.generate_report()
            else:
                return {"error": f"Неизвестный инструмент: {tool_name}"}
        except Exception as e:
            return {"error": str(e)}

# ============================================================================
# 8) ГЛОБАЛЬНЫЕ ОБЪЕКТЫ
# ============================================================================

ws = Workspace()
bus = MessageBus()
tools = Tools(ws)

# ============================================================================
# 9) АГЕНТ С ДВУХЭТАПНОЙ ГЕНЕРАЦИЕЙ КОДА
# ============================================================================

class LlmAgent:
    def __init__(self, name: str, system_role: str, allowed_tools: List[str], description: str):
        self.name = name
        self.system_role = system_role
        self.allowed_tools = allowed_tools
        self.description = description

    def get_context(self, goal: str, state: WorkflowState) -> str:
        """Формирует контекст для принятия решения"""
        recent = bus.get_last_messages(8)
        history = "\n".join([
            f"[{m.sender}→{m.recipient}]: {m.content[:150]}"
            for m in recent
        ])

        files_info = f"Файлы: {list(ws.files.keys())}" if ws.files else "Файлов нет"
        tests_info = f"Тесты: {list(ws.test_results.keys())}" if ws.test_results else "Тестов нет"

        return f"""ЦЕЛЬ: {goal}

СОСТОЯНИЕ: {state.value}
{files_info}
{tests_info}

ИСТОРИЯ:
{history}

ВАШ ИНСТРУМЕНТАРИЙ: {', '.join(self.allowed_tools) if self.allowed_tools else 'нет'}"""

    def _generate_code_directly(self, filename: str, spec: str) -> Optional[str]:
        """Генерирует код напрямую без JSON обёртки"""
        print(f"   🔧 [{self.name}] Генерирую код для {filename} напрямую...")

        prompt = f"""Напиши Python код для файла {filename}.

Спецификация:
{spec}

ВАЖНО:
- Верни ТОЛЬКО чистый Python код
- БЕЗ markdown разметки (без ```python)
- БЕЗ пояснений до или после кода
- Начни сразу с кода (импорты, классы, функции)"""

        code = call_llm_simple(
            prompt=prompt,
            system_prompt="Ты опытный Python разработчик. Возвращай только чистый, рабочий код."
        )

        # Очистка от markdown если модель всё же добавила
        code = code.strip()
        if code.startswith("```python"):
            code = code[9:]
        elif code.startswith("```"):
            code = code[3:]
        if code.endswith("```"):
            code = code[:-3]
        code = code.strip()

        return code if code else None

    def _generate_tests_directly(self, filename: str, main_code: str) -> Optional[str]:
        """Генерирует тесты напрямую"""
        print(f"   🧪 [{self.name}] Генерирую тесты для {filename} напрямую...")

        prompt = f"""Напиши unit-тесты для следующего кода:

```python
{main_code}
```

Требования:
- Используй assert для проверок (не unittest)
- Проверь все методы класса
- Проверь обработку ошибок (деление на ноль, отрицательные числа)
- В конце добавь print("All tests passed!") если все тесты прошли

ВАЖНО: Верни ТОЛЬКО Python код тестов, БЕЗ markdown."""

        test_code = call_llm_simple(
            prompt=prompt,
            system_prompt="Ты опытный тестировщик. Пиши простые, понятные тесты с assert."
        )

        # Очистка
        test_code = test_code.strip()
        if test_code.startswith("```python"):
            test_code = test_code[9:]
        elif test_code.startswith("```"):
            test_code = test_code[3:]
        if test_code.endswith("```"):
            test_code = test_code[:-3]

        return test_code.strip() if test_code.strip() else None

    def decide(self, goal: str, state: WorkflowState) -> AgentActionFlat:
        """Принимает решение о следующем действии"""
        context = self.get_context(goal, state)

        # Специальная логика для coder - двухэтапная генерация
        if self.name == "coder" and state == WorkflowState.CODING:
            if "calculator.py" not in ws.files:
                return self._handle_coding_task(goal)
            elif "test_calculator.py" not in ws.files:
                return self._handle_test_creation()

        # Специальная логика для tester
        if self.name == "tester" and state == WorkflowState.TESTING:
            if "calculator.py" in ws.files and "calculator.py" not in ws.test_results:
                return self._handle_testing_task()

        # Стандартный путь через LLM
        return self._decide_via_llm(goal, state, context)

    def _handle_coding_task(self, goal: str) -> AgentActionFlat:
        """Обработка задачи кодирования"""
        code = self._generate_code_directly("calculator.py", goal)

        if code:
            result = ws.store_code("calculator.py", code)
            if result.get("success"):
                return AgentActionFlat(
                    action="reply",
                    content=f"✅ Создал calculator.py ({result['lines']} строк). Готов к тестированию."
                )

        return AgentActionFlat(
            action="reply",
            content="❌ Не удалось сгенерировать код. Нужна помощь."
        )

    def _handle_test_creation(self) -> AgentActionFlat:
        """Создание тестов"""
        main_code = ws.files.get("calculator.py", "")
        test_code = self._generate_tests_directly("calculator.py", main_code)

        if test_code:
            result = ws.store_code("test_calculator.py", test_code)
            if result.get("success"):
                return AgentActionFlat(
                    action="reply",
                    content=f"✅ Создал test_calculator.py ({result['lines']} строк). Передаю тестировщику."
                )

        return AgentActionFlat(
            action="reply",
            content="❌ Не удалось создать тесты."
        )

    def _handle_testing_task(self) -> AgentActionFlat:
        """Запуск тестов"""
        test_code = ws.files.get("test_calculator.py", "")

        if not test_code:
            # Генерируем тесты если их нет
            main_code = ws.files.get("calculator.py", "")
            test_code = self._generate_tests_directly("calculator.py", main_code)
            if test_code:
                ws.store_code("test_calculator.py", test_code)

        if test_code:
            result = tools.call("run_tests", filename="calculator.py", test_code=test_code)

            if result.get("passed"):
                return AgentActionFlat(
                    action="reply",
                    content=f"✅ Все тесты пройдены!\n{result.get('stdout', '')[:200]}"
                )
            else:
                return AgentActionFlat(
                    action="reply",
                    content=f"❌ Тесты не прошли: {result.get('error', 'Unknown')[:200]}"
                )

        return AgentActionFlat(
            action="ask_agent",
            target="coder",
            message="Нужны тесты для calculator.py"
        )

    def _decide_via_llm(self, goal: str, state: WorkflowState, context: str) -> AgentActionFlat:
        """Стандартное решение через LLM"""

        examples = '''ПРИМЕРЫ JSON:

1. Попросить агента:
{"action": "ask_agent", "target": "coder", "message": "Создай файл X", "tool_name": "", "filename": "", "code_base64": "", "test_code_base64": "", "content": "", "summary": ""}

2. Ответить/сообщить:
{"action": "reply", "target": "none", "message": "", "tool_name": "", "filename": "", "code_base64": "", "test_code_base64": "", "content": "Задача выполнена", "summary": ""}

3. Завершить:
{"action": "finish", "target": "none", "message": "", "tool_name": "", "filename": "", "code_base64": "", "test_code_base64": "", "content": "", "summary": "Все задачи выполнены успешно"}

4. Использовать read_code:
{"action": "use_tool", "target": "none", "message": "", "tool_name": "read_code", "filename": "calculator.py", "code_base64": "", "test_code_base64": "", "content": "", "summary": ""}

5. Использовать analyze_code_quality:
{"action": "use_tool", "target": "none", "message": "", "tool_name": "analyze_code_quality", "filename": "calculator.py", "code_base64": "", "test_code_base64": "", "content": "", "summary": ""}'''

        system_prompt = f"""Вы — {self.system_role}.
Роль: {self.description}
Доступные инструменты: {', '.join(self.allowed_tools) if self.allowed_tools else 'нет'}

ПРАВИЛА:
1. Возвращайте ТОЛЬКО JSON, ничего больше
2. Все 8 полей обязательны
3. Неиспользуемые поля = пустая строка ""
4. target для ask_agent: planner/coder/tester/reviewer
5. target для остальных: "none"

{examples}"""

        prompt = f"""{context}

Текущее состояние: {state.value}
Что вы делаете? Верните JSON:"""

        try:
            return call_llm_with_schema(
                prompt=prompt,
                response_schema=AgentActionFlat,
                system_prompt=system_prompt,
                strict=True
            )
        except Exception as e:
            print(f"   ⚠️ [{self.name}] LLM ошибка: {e}")
            return self._default_action(state)

    def _default_action(self, state: WorkflowState) -> AgentActionFlat:
        """Действие по умолчанию"""
        defaults = {
            "planner": AgentActionFlat(action="ask_agent", target="coder", message="Начни реализацию"),
            "coder": AgentActionFlat(action="reply", content="Работаю над кодом..."),
            "tester": AgentActionFlat(action="ask_agent", target="coder", message="Нужен код для тестов"),
            "reviewer": AgentActionFlat(action="reply", content="Ожидаю код для ревью"),
        }
        return defaults.get(self.name, AgentActionFlat(action="reply", content="Ожидание..."))

# ============================================================================
# 10) ОРКЕСТРАТОР С STATE MACHINE
# ============================================================================

class Orchestrator:
    def __init__(self):
        self.agents = {
            "planner": LlmAgent(
                name="planner",
                system_role="Архитектор",
                allowed_tools=[],
                description="Планирует работу и координирует команду"
            ),
            "coder": LlmAgent(
                name="coder",
                system_role="Разработчик",
                allowed_tools=["store_code", "read_code", "run_python"],
                description="Пишет чистый, документированный Python код"
            ),
            "tester": LlmAgent(
                name="tester",
                system_role="Тестировщик",
                allowed_tools=["read_code", "run_tests"],
                description="Пишет и запускает unit-тесты"
            ),
            "reviewer": LlmAgent(
                name="reviewer",
                system_role="Ревьювер",
                allowed_tools=["read_code", "analyze_code_quality", "generate_report"],
                description="Проводит code review и оценивает качество"
            )
        }

        self.state = WorkflowState.PLANNING
        self.completed_files: set = set()
        self.tested_files: set = set()
        self.reviewed_files: set = set()
        self.required_files = {"calculator.py"}

    def update_state(self):
        """Обновляет состояние workflow"""
        if not self.completed_files:
            self.state = WorkflowState.CODING
        elif not self.required_files.issubset(self.completed_files):
            self.state = WorkflowState.CODING
        elif not self.tested_files:
            self.state = WorkflowState.TESTING
        elif not self.reviewed_files:
            self.state = WorkflowState.REVIEWING
        else:
            self.state = WorkflowState.DONE

        print(f"   📊 Состояние: {self.state.value}")

    def process_action(self, agent_name: str, action: AgentActionFlat) -> bool:
        """Обрабатывает действие агента. Возвращает True если workflow завершён."""

        # Валидация: нельзя писать самому себе
        if action.action == "ask_agent" and action.target == agent_name:
            print(f"   ⚠️ [{agent_name}] Нельзя писать самому себе!")
            return False

        if action.action == "ask_agent":
            if action.target not in self.agents or action.target == "none":
                print(f"   ⚠️ [{agent_name}] Неверный target: {action.target}")
                return False

            bus.send(BusMessage(
                sender=agent_name,
                recipient=action.target,
                content=action.message
            ))
            return False

        elif action.action == "use_tool":
            allowed = self.agents[agent_name].allowed_tools
            if action.tool_name not in allowed:
                print(f"   ⛔ [{agent_name}] Нет доступа к {action.tool_name}")
                return False

            # Собираем аргументы
            kwargs = {}
            if action.filename:
                kwargs["filename"] = action.filename
            if action.code_base64:
                kwargs["code"] = action.get_decoded_code()
            if action.test_code_base64:
                kwargs["test_code"] = action.get_decoded_test_code()

            result = tools.call(action.tool_name, **kwargs)
            result_str = json.dumps(result, ensure_ascii=False)[:300]

            bus.send(BusMessage(
                sender=agent_name,
                recipient="broadcast",
                content=f"[{action.tool_name}]: {result_str}"
            ))

            print(f"   🔧 [{agent_name}] {action.tool_name}: {result_str[:100]}...")

            # Обновляем состояние
            if action.tool_name == "store_code" and result.get("success"):
                self.completed_files.add(result.get("filename", ""))
                self.update_state()

            if action.tool_name == "run_tests" and result.get("passed"):
                self.tested_files.add(result.get("file", ""))
                self.update_state()

            if action.tool_name == "analyze_code_quality":
                self.reviewed_files.add(action.filename)
                self.update_state()

            return False

        elif action.action == "reply":
            bus.send(BusMessage(
                sender=agent_name,
                recipient="broadcast",
                content=action.content
            ))
            print(f"   💬 [{agent_name}]: {action.content[:100]}...")

            # Проверяем завершение этапов по содержимому
            if "calculator.py" in action.content and ("создал" in action.content.lower() or "✅" in action.content):
                self.completed_files.add("calculator.py")
                self.update_state()

            if "тесты пройдены" in action.content.lower() or "all tests passed" in action.content.lower():
                self.tested_files.add("calculator.py")
                self.update_state()

            return False

        elif action.action == "finish":
            print(f"   ✅ [{agent_name}] ЗАВЕРШЕНО: {action.summary}")
            bus.send(BusMessage(
                sender=agent_name,
                recipient="broadcast",
                content=f"FINISH: {action.summary}"
            ))
            return True

        return False

    def step(self, goal: str, round_num: int) -> bool:
        """Выполняет один раунд"""
        print(f"\n{'='*50}\n🔄 РАУНД {round_num} | Состояние: {self.state.value}\n{'='*50}")

        # Определяем порядок агентов в зависимости от состояния
        if self.state == WorkflowState.PLANNING:
            order = ["planner"]
        elif self.state == WorkflowState.CODING:
            order = ["coder"]
        elif self.state == WorkflowState.TESTING:
            order = ["tester"]
        elif self.state == WorkflowState.REVIEWING:
            order = ["reviewer"]
        else:
            order = ["reviewer"]  # Финальное завершение

        for agent_name in order:
            print(f"\n🔹 Агент: {agent_name.upper()}")

            # Получаем входящие
            inbox = bus.receive_for(agent_name)
            if inbox:
                print(f"   📥 Входящие ({len(inbox)}):")
                for m in inbox[-2:]:
                    print(f"      [{m.sender}]: {m.content[:60]}...")

            # Агент принимает решение
            action = self.agents[agent_name].decide(goal, self.state)
            print(f"   Действие: {action.action}")

            # Обрабатываем
            if self.process_action(agent_name, action):
                return True  # Завершение

            # Обновляем состояние после каждого агента
            self.update_state()

            # Проверяем на DONE
            if self.state == WorkflowState.DONE:
                print("\n🎉 Workflow завершён!")
                return True

        return False

    def run(self, goal: str, max_rounds: int = 15):
        """Запуск системы"""
        print(f"\n{'='*60}")
        print("🚀 ЗАПУСК МУЛЬТИАГЕНТНОЙ СИСТЕМЫ v2.0")
        print(f"{'='*60}")
        print(f"\n📋 Цель:\n{goal[:300]}...\n")

        bus.send(BusMessage(
            sender="orchestrator",
            recipient="broadcast",
            content=f"СТАРТ. Цель: {goal[:150]}"
        ))

        self.update_state()

        for round_num in range(1, max_rounds + 1):
            if self.step(goal, round_num):
                break
        else:
            print(f"\n⏱️ Лимит раундов ({max_rounds}) достигнут")

        # Финальный отчёт
        self._print_final_report()

    def _print_final_report(self):
        """Выводит финальный отчёт"""
        print(f"\n{'='*60}")
        print("📊 ФИНАЛЬНЫЙ ОТЧЁТ")
        print(f"{'='*60}")

        print(f"\n📁 Файлы ({len(ws.files)}):")
        for filename, code in ws.files.items():
            print(f"\n{'─'*40}")
            print(f"📄 {filename} ({len(code.splitlines())} строк)")
            print(f"{'─'*40}")
            print(code)

        print(f"\n🧪 Результаты тестов:")
        if ws.test_results:
            for name, res in ws.test_results.items():
                status = "✅ PASSED" if res.get("passed") else "❌ FAILED"
                print(f"   {name}: {status}")
                if res.get("stdout"):
                    print(f"   Вывод: {res['stdout'][:150]}")
                if res.get("error"):
                    print(f"   Ошибка: {res['error'][:150]}")
        else:
            print("   Тесты не запускались")

        print(f"\n🔍 Code Review:")
        if ws.code_reviews:
            for name, review in ws.code_reviews.items():
                print(f"   {name}: {review.get('quality_score', 0)}/100")
                if review.get("recommendations"):
                    for rec in review["recommendations"]:
                        print(f"      • {rec}")
        else:
            print("   Ревью не проводилось")

        print(f"\n{'='*60}")
        print("✅ Работа системы завершена")
        print(f"{'='*60}")

# ============================================================================
# 11) ЗАПУСК
# ============================================================================

GOAL = """
Реализовать класс Calculator для математических операций:
- Метод add(x, y): сложение двух чисел
- Метод subtract(x, y): вычитание
- Метод multiply(x, y): умножение
- Метод divide(x, y): деление с проверкой деления на ноль
- Метод power(x, y): возведение в степень
- Метод sqrt(x): квадратный корень с проверкой отрицательного числа

Требования:
- Код должен быть чистым и хорошо документированным
- Класс должен обрабатывать ошибки (деление на ноль, отрицательные числа для корня)
- Написать unit-тесты для всех методов
- Провести code review и сгенерировать отчет
"""


orchestrator = Orchestrator()
orchestrator.run(goal=GOAL, max_rounds=15)



🚀 ЗАПУСК МУЛЬТИАГЕНТНОЙ СИСТЕМЫ v2.0

📋 Цель:

Реализовать класс Calculator для математических операций:
- Метод add(x, y): сложение двух чисел
- Метод subtract(x, y): вычитание
- Метод multiply(x, y): умножение  
- Метод divide(x, y): деление с проверкой деления на ноль
- Метод power(x, y): возведение в степень
- Метод sqrt(x): квадратный коре...

📨 [orchestrator → broadcast]: СТАРТ. Цель: 
Реализовать класс Calculator для математических операций:
- Метод ...
   📊 Состояние: coding

🔄 РАУНД 1 | Состояние: coding

🔹 Агент: CODER
   📥 Входящие (1):
      [orchestrator]: СТАРТ. Цель: 
Реализовать класс Calculator для математически...
   🔧 [coder] Генерирую код для calculator.py напрямую...
   Действие: reply
📨 [coder → broadcast]: ✅ Создал calculator.py (275 строк). Готов к тестированию....
   💬 [coder]: ✅ Создал calculator.py (275 строк). Готов к тестированию....
   📊 Состояние: testing
   📊 Состояние: testing

🔄 РАУНД 2 | Состояние: testing

🔹 Агент: TESTER
   📥 Входящие (1):
      [c

In [22]:
import math
import unittest
from typing import Any


class Calculator:
    """
    A simple calculator class providing basic mathematical operations.

    Methods
    -------
    add(x, y)
        Return the sum of x and y.
    subtract(x, y)
        Return the difference x - y.
    multiply(x, y)
        Return the product of x and y.
    divide(x, y)
        Return the quotient x / y. Raises ZeroDivisionError if y is zero.
    power(x, y)
        Return x raised to the power y.
    sqrt(x)
        Return the square root of x. Raises ValueError if x is negative.
    """

    @staticmethod
    def _validate_number(value: Any, name: str = "value") -> float:
        """
        Validate that value is an int or float and return it as float.

        Raises
        ------
        TypeError
            If value is not a numeric type.
        """
        if isinstance(value, (int, float)):
            return float(value)
        raise TypeError(f"{name} must be a number (int or float), got {type(value).__name__}")

    def add(self, x: Any, y: Any) -> float:
        """
        Return the sum of x and y.

        Raises
        ------
        TypeError
            If x or y is not numeric.
        """
        x_val = self._validate_number(x, "x")
        y_val = self._validate_number(y, "y")
        result = x_val + y_val
        return result

    def subtract(self, x: Any, y: Any) -> float:
        """
        Return the difference x - y.

        Raises
        ------
        TypeError
            If x or y is not numeric.
        """
        x_val = self._validate_number(x, "x")
        y_val = self._validate_number(y, "y")
        result = x_val - y_val
        return result

    def multiply(self, x: Any, y: Any) -> float:
        """
        Return the product of x and y.

        Raises
        ------
        TypeError
            If x or y is not numeric.
        """
        x_val = self._validate_number(x, "x")
        y_val = self._validate_number(y, "y")
        result = x_val * y_val
        return result

    def divide(self, x: Any, y: Any) -> float:
        """
        Return the quotient x / y.

        Raises
        ------
        TypeError
            If x or y is not numeric.
        ZeroDivisionError
            If y is zero.
        """
        x_val = self._validate_number(x, "x")
        y_val = self._validate_number(y, "y")
        if y_val == 0.0:
            raise ZeroDivisionError("Division by zero is not allowed.")
        result = x_val / y_val
        return result

    def power(self, x: Any, y: Any) -> float:
        """
        Return x raised to the power y.

        Raises
        ------
        TypeError
            If x or y is not numeric.
        """
        x_val = self._validate_number(x, "x")
        y_val = self._validate_number(y, "y")
        # Use Python's exponentiation which handles many cases (including negative exponents).
        result = x_val ** y_val
        return result

    def sqrt(self, x: Any) -> float:
        """
        Return the square root of x.

        Raises
        ------
        TypeError
            If x is not numeric.
        ValueError
            If x is negative.
        """
        x_val = self._validate_number(x, "x")
        if x_val < 0.0:
            raise ValueError("Square root of negative number is not defined for real results.")
        result = math.sqrt(x_val)
        return result


# Unit tests for Calculator
class TestCalculator(unittest.TestCase):
    def setUp(self):
        self.calc = Calculator()

    # add
    def test_add_integers(self):
        self.assertEqual(self.calc.add(2, 3), 5.0)

    def test_add_floats(self):
        self.assertAlmostEqual(self.calc.add(2.5, 3.1), 5.6)

    def test_add_type_error(self):
        with self.assertRaises(TypeError):
            self.calc.add("2", 3)

    # subtract
    def test_subtract(self):
        self.assertEqual(self.calc.subtract(10, 4), 6.0)

    def test_subtract_negative_result(self):
        self.assertEqual(self.calc.subtract(3, 5), -2.0)

    def test_subtract_type_error(self):
        with self.assertRaises(TypeError):
            self.calc.subtract(5, None)

    # multiply
    def test_multiply(self):
        self.assertEqual(self.calc.multiply(3, 4), 12.0)

    def test_multiply_by_zero(self):
        self.assertEqual(self.calc.multiply(100, 0), 0.0)

    def test_multiply_type_error(self):
        with self.assertRaises(TypeError):
            self.calc.multiply([], 3)

    # divide
    def test_divide(self):
        self.assertEqual(self.calc.divide(10, 2), 5.0)

    def test_divide_float_result(self):
        self.assertAlmostEqual(self.calc.divide(7, 2), 3.5)

    def test_divide_by_zero(self):
        with self.assertRaises(ZeroDivisionError):
            self.calc.divide(1, 0)

    def test_divide_type_error(self):
        with self.assertRaises(TypeError):
            self.calc.divide(1, "0")

    # power
    def test_power_positive(self):
        self.assertEqual(self.calc.power(2, 3), 8.0)

    def test_power_negative_exponent(self):
        self.assertAlmostEqual(self.calc.power(2, -1), 0.5)

    def test_power_zero_exponent(self):
        self.assertEqual(self.calc.power(5, 0), 1.0)

    def test_power_type_error(self):
        with self.assertRaises(TypeError):
            self.calc.power("2", 3)

    # sqrt
    def test_sqrt_positive(self):
        self.assertEqual(self.calc.sqrt(9), 3.0)

    def test_sqrt_zero(self):
        self.assertEqual(self.calc.sqrt(0), 0.0)

    def test_sqrt_negative(self):
        with self.assertRaises(ValueError):
            self.calc.sqrt(-4)

    def test_sqrt_type_error(self):
        with self.assertRaises(TypeError):
            self.calc.sqrt([9])


def run_tests() -> unittest.result.TestResult:
    """
    Run unit tests for this module programmatically and return the result.
    """
    loader = unittest.TestLoader()
    suite = loader.loadTestsFromTestCase(TestCalculator)
    runner = unittest.TextTestRunner(verbosity=2)
    result = runner.run(suite)
    return result


def generate_code_review_report() -> str:
    """
    Generate a static code review report for the Calculator implementation.

    The report covers design, correctness, error handling, testing, and suggestions.
    """
    report_lines = [
        "Code Review Report for Calculator",
        "---------------------------------",
        "Summary:",
        " - The Calculator class implements basic arithmetic operations: add, subtract, multiply, divide, power, sqrt.",
        " - Input validation is present: non-numeric inputs raise TypeError.",
        " - Edge cases are handled: division by zero raises ZeroDivisionError; sqrt of negative raises ValueError.",
        "",
        "Strengths:",
        " - Clear separation of concerns: a private _validate_number helper centralizes input validation.",
        " - Methods are small and focused with explicit docstrings and type hints.",
        " - Unit tests cover normal cases and expected error conditions for all methods.",
        " - Tests are run programmatically, producing clear output when the module is executed as a script.",
        "",
        "Potential Improvements:",
        " - Consider supporting Decimal for higher-precision arithmetic if needed by consumers.",
        " - For power with non-integer exponents and negative bases, results may be complex; consider documenting behavior or returning complex numbers if desired.",
        " - Input validation currently accepts ints and floats only. If you want to accept types implementing the numeric protocol (e.g., numpy types), adapt validation accordingly.",
        " - Add property-based tests (hypothesis) to increase confidence for a wider range of inputs.",
        "",
        "Style and Maintainability:",
        " - Docstrings are present; consider adding examples in docstrings for the public methods.",
        " - The code is concise and follows common Pythonic patterns.",
        "",
        "Test Coverage Observations:",
        " - Unit tests cover typical positive, negative, and error scenarios for each method.",
        " - Consider adding tests for very large numbers, infinities, and NaN handling if these are relevant.",
        "",
        "Conclusion:",
        " - The implementation is correct and robust for common use cases.",
        " - The suggestions above can be considered for expanded functionality or stricter numeric requirements.",
    ]
    return "\n".join(report_lines)


# if __name__ == "__main__":
# Run tests and then print the code review report.
test_result = run_tests()
print()
print(generate_code_review_report())
# Exit with non-zero status if tests failed or errored.
if not test_result.wasSuccessful():
    raise SystemExit(1)

test_add_floats (__main__.TestCalculator.test_add_floats) ... ok
test_add_integers (__main__.TestCalculator.test_add_integers) ... ok
test_add_type_error (__main__.TestCalculator.test_add_type_error) ... ok
test_divide (__main__.TestCalculator.test_divide) ... ok
test_divide_by_zero (__main__.TestCalculator.test_divide_by_zero) ... ok
test_divide_float_result (__main__.TestCalculator.test_divide_float_result) ... ok
test_divide_type_error (__main__.TestCalculator.test_divide_type_error) ... ok
test_multiply (__main__.TestCalculator.test_multiply) ... ok
test_multiply_by_zero (__main__.TestCalculator.test_multiply_by_zero) ... ok
test_multiply_type_error (__main__.TestCalculator.test_multiply_type_error) ... ok
test_power_negative_exponent (__main__.TestCalculator.test_power_negative_exponent) ... ok
test_power_positive (__main__.TestCalculator.test_power_positive) ... ok
test_power_type_error (__main__.TestCalculator.test_power_type_error) ... ok
test_power_zero_exponent (__main__.Test


Code Review Report for Calculator
---------------------------------
Summary:
 - The Calculator class implements basic arithmetic operations: add, subtract, multiply, divide, power, sqrt.
 - Input validation is present: non-numeric inputs raise TypeError.
 - Edge cases are handled: division by zero raises ZeroDivisionError; sqrt of negative raises ValueError.

Strengths:
 - Clear separation of concerns: a private _validate_number helper centralizes input validation.
 - Methods are small and focused with explicit docstrings and type hints.
 - Unit tests cover normal cases and expected error conditions for all methods.
 - Tests are run programmatically, producing clear output when the module is executed as a script.

Potential Improvements:
 - Consider supporting Decimal for higher-precision arithmetic if needed by consumers.
 - For power with non-integer exponents and negative bases, results may be complex; consider documenting behavior or returning complex numbers if desired.
 - Input 